# Gurobi optimization using multiple machine learning models
## Optimize for Price and Supply of Avocados - time horizon

THIS IS THE SAME OPTIMIZATION PROBLEM OF NOTEBOOK 2 BUT WITH THE DIFFERENCE TO CONSIDERING A TIME HORIZON PLANNING. SO OPTIMIZATION PROBLEM IS RE DEFINED TO ADD TIME BUT THE MACHINE LEARNINGS MODELS OF NOTEBOOKS 2 ARE THE SAME.
**SO, The ml models predict to one time horizon (and it is the same of notebook 2), but the optimization model consider the time horizon**

**DOCUMENTATION**
- In this example there multiple linear regressions, but gurobi machine learning acept multiple models. Documentation **"gurobi-machinelearning"**

https://gurobi-machinelearning.readthedocs.io/en/stable/api.html


- In addition, to define the decision variables, parameters, restriction, etc of the optimization model are used **"gurobipy-pandas"**. Using this package is possible define the optimization model using pandas DataFrames

https://gurobipy-pandas.readthedocs.io/en/stable/

In [ ]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Root repo

In [ ]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-1]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

## RUN OPTIMIZATION

## 0. Model Gurobi

In [ ]:
# env = gp.Env(params=params)

#Create the model within the Gurobi environment
model_opt = gp.Model(name = "test model")

## 1. Define set

In [ ]:
############## set regions - index pandas ##############

# generate a pandas index with the values of the regions. This works as sets of optimization model
list_regions = ['Great_Lakes',
                'Midsouth',
                'Northeast',
                'Northern_New_England',
                'Plains',
                'SouthCentral',
                'Southeast',
                'West']
regions = list_regions

# index pandas
index_regions = pd.Index(regions)
index_regions

In [ ]:
############## set time - index pandas ##############

# generate a pandas index with time horizon planning
time_horizon_planning = ['t1', 't2', 't3', 't4', 't5']

# index pandas
index_time = pd.Index(time_horizon_planning)
index_time

In [ ]:
############## multi set region&time - index pandas ##############

#index pandas
index_region_time = pd.MultiIndex.from_product((regions, time_horizon_planning), 
                                               names = ('region', 'time')
                                              )
index_region_time

## 2. Create decision variables - one set - multisets

### 2.1 Create decision variable set region (one set)

In [ ]:
# test create decision variable set region
var_test_region = gppd.add_vars(model_opt, index_regions, name = "var test")
var_test_region

In [ ]:
# filter decision variable element of set region
var_test_region['Southeast']

### 2.2 Create decision variable set time (one set)

In [ ]:
# prueba crear variable set time
var_test_time = gppd.add_vars(model_opt, index_time, name = "var test")
var_test_time

In [ ]:
# filter decision variable element of set time
var_test_time['t3']

### 2.3 Create decision variable multi sets (region, time) (pandas multiindex)
IMPORTANT - TO FILTER INDEX IT IS NECESARY USE .LOC

In [ ]:
# prueba crear variable decision multi conjuntos
var_test_region_time = gppd.add_vars(model_opt, index_region_time, name = "var test")
var_test_region_time

In [ ]:
# filter one element set region and time

#var_test_region_time['Southeast']['t1'] # ex1
#var_test_region_time['Southeast']['t2'] # ex2
#var_test_region_time['Plains']['t4'] #ex3

var_test_region_time.loc['Plains', 't4']  # correct example using loc[set, set]

In [ ]:
#### filter one element set one /// all elements set two

# var_test_region_time['Southeast'][:] # incorrect example

var_test_region_time.loc['Plains', :] # correct example using loc[set, set]

In [ ]:
# filter all elements set one // one element set two
var_test_region_time.loc[:, 't4']

## 3. Sum elements across sets in decision variables

In [ ]:
# Define decision variable in this sets
var_test_sum = gppd.add_vars(model_opt, index_region_time, name = "var_test")
model_opt.update()

In [ ]:
var_test_sum

### 3.1. Sum all elements in decision variable (sum across all sets) (USING .sum and quicksum)

In [ ]:
# .sum()
sum_sum = var_test_sum.sum()
sum_sum

In [ ]:
# quicksum

from gurobipy import quicksum
sum_in_r_t_quicksum = quicksum(var_test_sum[r, t] for r in index_regions for t in index_time)
sum_in_r_t_quicksum

### 3.2 Sum elements only across ONE set. Sum across set regions (USING quicksum)

In [ ]:
# sum region for each time
sum_in_r_quicksum = quicksum(var_test_sum[r] for r in index_regions)
sum_in_r_quicksum

In [ ]:
# show value in one region
sum_in_r_quicksum['t1']

### 3.3 Sum elements only across ONE set. Sum across set regions (USING groupby and sum)
Docu: https://gurobipy-pandas.readthedocs.io/en/latest/usage.html

In [ ]:
# "grouby by" by the set that I don't want to sum
sum_in_r_groupsum = var_test_sum.groupby('time').sum()
sum_in_r_groupsum

In [ ]:
sum_in_r_groupsum['t2']

## 4. Define a constraint for all t in Time

In [ ]:
# define right side values of the restriction
np.random.seed(42)
rs = np.random.random(len(time_horizon_planning))
rs

In [ ]:
# define a constraint
constraint_for_all_time = gppd.add_constrs(model_opt, sum_in_r_groupsum, gp.GRB.EQUAL, rs)
model_opt.update()

In [ ]:
# show constraint but it is useless
constraint_for_all_time

### 5. Define a constraint of previous time
SOLDS <= SUPPLY + INVENTARY (t previous)

\begin{align*} 
s_r(t) &\leq x_r(t) + I_r(t-1)               \:\:\:\:\forall r \forall t\\
\end{align*}

In [ ]:
x = gppd.add_vars(model_opt, index_region_time, name = 'supply')
x

In [ ]:
# manually it is necesary to define. Next use a for to run across all the set
x.loc['Great_Lakes', 't2'] - x.loc['Great_Lakes', 't1']